## Importing Packages

In [34]:
import pandas as pd
import numpy as np #import numpy to enable array functionality
from sklearn.pipeline import Pipeline #let's import the pipeline functionality
from sklearn.feature_extraction.text import CountVectorizer #and we will import a simple pre-processing method
from sklearn.feature_extraction.text import TfidfTransformer #and a representation learner
from sklearn.neighbors import KNeighborsClassifier #and a simple classifier model
from sklearn.model_selection import StratifiedKFold #cross fold is sometimes called k-fold. Calling the stratified version ensures that classes have equal representation across folds
from sklearn.metrics import accuracy_score #import an accuracy metric to tell us how well the model is doing
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lapos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lapos\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## Loading the data
### the data that we are using comes split into 3 formats
 1. train
 2. test
 3. unsupervised



In [25]:

df_train = pd.read_parquet("hf://datasets/stanfordnlp/imdb/plain_text/train-00000-of-00001.parquet")
df_test = pd.read_parquet("hf://datasets/stanfordnlp/imdb/plain_text/test-00000-of-00001.parquet")
df_unsupervised = pd.read_parquet("hf://datasets/stanfordnlp/imdb/plain_text/unsupervised-00000-of-00001.parquet")


In [28]:
df_train.shape

(25000, 2)

In [31]:
random_elements = df_train['text'].sample(10)

In [32]:
random_elements

22543    It's heart-warming to see a movie that doesn't...
10708    Don't get fooled with all the big names like B...
9714     I'm guessing the writers have never read a boo...
16938    Ang Lee clearly likes to ease into a film, to ...
382      I love Jamie Foxx.<br /><br />And I enjoy 99% ...
3573     A slasher flick, made in the early 80's, has a...
2420     Years have gone by since Don Wilson used his m...
23559    <br /><br />I saw The Glacier Fox in the theat...
8591     If the only sex you've ever had is with a farm...
14836    This was a hit in the South By Southwest (SXSW...
Name: text, dtype: object

### We print random texts to see exaclty how the data looks like so we can preprocess it


In [33]:
for s in random_elements:
    print(s)
    print("-----------")

It's heart-warming to see a movie that doesn't bash males. In this one the wife/mother leaves her family to "get in touch" with herself - or pursue her libido. The father stays with and nurtures the kids, letting neither his work nor his love life interfere with his love of and responsibility to them.
-----------
Don't get fooled with all the big names like Burt Reynolds,James Woods and Anne Archer. They are just glorified extra's. Their scenes were probably filmed in one day or so. Whatever their motives for being in this movie, if you have an actor like James Woods you better make good use of him. To me this is a sign of bad direction through and through. The plot itself wasn't that bad. And the acting from most of the actors was above average. Cuba Gooding Jr. however was terrible. He was so unbelievable that I almost laughed at his dramatic scenes. And since this was meant as a serious movie that can't be a good thing. The action scenes were not bad,but they lacked that special pun

In [39]:
def remove_html_tags(text):
    """Remove html tags from a text"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', s)
# we test it on our random elements to make sure it works as intended
random_elements_v1 = []
for s in random_elements:
    print(remove_html_tags(s))
    print("-----------")
    random_elements_v1.append(remove_html_tags(s))


It's heart-warming to see a movie that doesn't bash males. In this one the wife/mother leaves her family to "get in touch" with herself - or pursue her libido. The father stays with and nurtures the kids, letting neither his work nor his love life interfere with his love of and responsibility to them.
-----------
Don't get fooled with all the big names like Burt Reynolds,James Woods and Anne Archer. They are just glorified extra's. Their scenes were probably filmed in one day or so. Whatever their motives for being in this movie, if you have an actor like James Woods you better make good use of him. To me this is a sign of bad direction through and through. The plot itself wasn't that bad. And the acting from most of the actors was above average. Cuba Gooding Jr. however was terrible. He was so unbelievable that I almost laughed at his dramatic scenes. And since this was meant as a serious movie that can't be a good thing. The action scenes were not bad,but they lacked that special pun

In [37]:
def tokenize_text(text):
    """Tokenize text"""
    tokens = word_tokenize(text, language='english')
    return tokens


In [43]:
random_elements_v2 = []
for s in random_elements_v1:
    print(tokenize_text(s))
    print("-----------")
    random_elements_v2.append(tokenize_text(s))

['It', "'s", 'heart-warming', 'to', 'see', 'a', 'movie', 'that', 'does', "n't", 'bash', 'males', '.', 'In', 'this', 'one', 'the', 'wife/mother', 'leaves', 'her', 'family', 'to', '``', 'get', 'in', 'touch', "''", 'with', 'herself', '-', 'or', 'pursue', 'her', 'libido', '.', 'The', 'father', 'stays', 'with', 'and', 'nurtures', 'the', 'kids', ',', 'letting', 'neither', 'his', 'work', 'nor', 'his', 'love', 'life', 'interfere', 'with', 'his', 'love', 'of', 'and', 'responsibility', 'to', 'them', '.']
-----------
['Do', "n't", 'get', 'fooled', 'with', 'all', 'the', 'big', 'names', 'like', 'Burt', 'Reynolds', ',', 'James', 'Woods', 'and', 'Anne', 'Archer', '.', 'They', 'are', 'just', 'glorified', 'extra', "'s", '.', 'Their', 'scenes', 'were', 'probably', 'filmed', 'in', 'one', 'day', 'or', 'so', '.', 'Whatever', 'their', 'motives', 'for', 'being', 'in', 'this', 'movie', ',', 'if', 'you', 'have', 'an', 'actor', 'like', 'James', 'Woods', 'you', 'better', 'make', 'good', 'use', 'of', 'him', '.', 

In [45]:
def remove_punctuation(text):
    """Remove punctuation from a text"""
    # Handle both strings and lists
    if isinstance(text, list):
        return [re.sub(r'[^\w\s]', '', item) if isinstance(item, str) else item for item in text]
    elif isinstance(text, str):
        return re.sub(r'[^\w\s]', '', text)
    else:
        return text

In [46]:
random_elements_v3 = []
for s in random_elements_v2:
    print(remove_punctuation(s))
    print("-----------")
    random_elements_v3.append(remove_punctuation(s))

['It', 's', 'heartwarming', 'to', 'see', 'a', 'movie', 'that', 'does', 'nt', 'bash', 'males', '', 'In', 'this', 'one', 'the', 'wifemother', 'leaves', 'her', 'family', 'to', '', 'get', 'in', 'touch', '', 'with', 'herself', '', 'or', 'pursue', 'her', 'libido', '', 'The', 'father', 'stays', 'with', 'and', 'nurtures', 'the', 'kids', '', 'letting', 'neither', 'his', 'work', 'nor', 'his', 'love', 'life', 'interfere', 'with', 'his', 'love', 'of', 'and', 'responsibility', 'to', 'them', '']
-----------
['Do', 'nt', 'get', 'fooled', 'with', 'all', 'the', 'big', 'names', 'like', 'Burt', 'Reynolds', '', 'James', 'Woods', 'and', 'Anne', 'Archer', '', 'They', 'are', 'just', 'glorified', 'extra', 's', '', 'Their', 'scenes', 'were', 'probably', 'filmed', 'in', 'one', 'day', 'or', 'so', '', 'Whatever', 'their', 'motives', 'for', 'being', 'in', 'this', 'movie', '', 'if', 'you', 'have', 'an', 'actor', 'like', 'James', 'Woods', 'you', 'better', 'make', 'good', 'use', 'of', 'him', '', 'To', 'me', 'this', '

In [47]:
def remove_stopwords(text):
    swr_doc = [token for token in text if token not in stopwords.words('english')] #list compression to remove any stopwords from our list
    return swr_doc


In [48]:
random_elements_v4 = []
for s in random_elements_v3:
    print(remove_stopwords(s))
    print("-----------")
    random_elements_v4.append(remove_stopwords(s))

['It', 'heartwarming', 'see', 'movie', 'nt', 'bash', 'males', '', 'In', 'one', 'wifemother', 'leaves', 'family', '', 'get', 'touch', '', '', 'pursue', 'libido', '', 'The', 'father', 'stays', 'nurtures', 'kids', '', 'letting', 'neither', 'work', 'love', 'life', 'interfere', 'love', 'responsibility', '']
-----------
['Do', 'nt', 'get', 'fooled', 'big', 'names', 'like', 'Burt', 'Reynolds', '', 'James', 'Woods', 'Anne', 'Archer', '', 'They', 'glorified', 'extra', '', 'Their', 'scenes', 'probably', 'filmed', 'one', 'day', '', 'Whatever', 'motives', 'movie', '', 'actor', 'like', 'James', 'Woods', 'better', 'make', 'good', 'use', '', 'To', 'sign', 'bad', 'direction', '', 'The', 'plot', 'nt', 'bad', '', 'And', 'acting', 'actors', 'average', '', 'Cuba', 'Gooding', 'Jr', 'however', 'terrible', '', 'He', 'unbelievable', 'I', 'almost', 'laughed', 'dramatic', 'scenes', '', 'And', 'since', 'meant', 'serious', 'movie', 'ca', 'nt', 'good', 'thing', '', 'The', 'action', 'scenes', 'bad', '', 'lacked', '

In [49]:
def remove_empty_elements(text):
    for element in text:
        if element == '':
            text.remove(element)
    return text

In [50]:
random_elements_v5 = []
for s in random_elements_v4:
    print(remove_empty_elements(s))
    print("-----------")
    random_elements_v5.append(remove_empty_elements(s))


['It', 'heartwarming', 'see', 'movie', 'nt', 'bash', 'males', 'In', 'one', 'wifemother', 'leaves', 'family', 'get', 'touch', 'pursue', 'libido', 'The', 'father', 'stays', 'nurtures', 'kids', 'letting', 'neither', 'work', 'love', 'life', 'interfere', 'love', 'responsibility', '']
-----------
['Do', 'nt', 'get', 'fooled', 'big', 'names', 'like', 'Burt', 'Reynolds', 'James', 'Woods', 'Anne', 'Archer', 'They', 'glorified', 'extra', 'Their', 'scenes', 'probably', 'filmed', 'one', 'day', 'Whatever', 'motives', 'movie', 'actor', 'like', 'James', 'Woods', 'better', 'make', 'good', 'use', 'To', 'sign', 'bad', 'direction', 'The', 'plot', 'nt', 'bad', 'And', 'acting', 'actors', 'average', 'Cuba', 'Gooding', 'Jr', 'however', 'terrible', 'He', 'unbelievable', 'I', 'almost', 'laughed', 'dramatic', 'scenes', 'And', 'since', 'meant', 'serious', 'movie', 'ca', 'nt', 'good', 'thing', 'The', 'action', 'scenes', 'bad', 'lacked', 'special', 'punch', 'make', 'exciting', 'Again', 'better', 'direction', 'need

In [53]:
def lowercase_words(text):
    for i in range(len(text)):
        text[i] = text[i].lower()
    return text

In [54]:
random_elements_v6 = []
for s in random_elements_v5:
    print(lowercase_words(s))
    print("-----------")
    random_elements_v6.append(lowercase_words(s))

['it', 'heartwarming', 'see', 'movie', 'nt', 'bash', 'males', 'in', 'one', 'wifemother', 'leaves', 'family', 'get', 'touch', 'pursue', 'libido', 'the', 'father', 'stays', 'nurtures', 'kids', 'letting', 'neither', 'work', 'love', 'life', 'interfere', 'love', 'responsibility']
-----------
['do', 'nt', 'get', 'fooled', 'big', 'names', 'like', 'burt', 'reynolds', 'james', 'woods', 'anne', 'archer', 'they', 'glorified', 'extra', 'their', 'scenes', 'probably', 'filmed', 'one', 'day', 'whatever', 'motives', 'movie', 'actor', 'like', 'james', 'woods', 'better', 'make', 'good', 'use', 'to', 'sign', 'bad', 'direction', 'the', 'plot', 'nt', 'bad', 'and', 'acting', 'actors', 'average', 'cuba', 'gooding', 'jr', 'however', 'terrible', 'he', 'unbelievable', 'i', 'almost', 'laughed', 'dramatic', 'scenes', 'and', 'since', 'meant', 'serious', 'movie', 'ca', 'nt', 'good', 'thing', 'the', 'action', 'scenes', 'bad', 'lacked', 'special', 'punch', 'make', 'exciting', 'again', 'better', 'direction', 'needed',

In [55]:
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    for i in range(len(text)):
        text[i] = lemmatizer.lemmatize(text[i])
    return text


In [56]:
random_elements_v7 = []
for s in random_elements_v6:
    print(lemmatize_words(s))
    print("-----------")
    random_elements_v7.append(lemmatize_words(s))

['it', 'heartwarming', 'see', 'movie', 'nt', 'bash', 'male', 'in', 'one', 'wifemother', 'leaf', 'family', 'get', 'touch', 'pursue', 'libido', 'the', 'father', 'stay', 'nurture', 'kid', 'letting', 'neither', 'work', 'love', 'life', 'interfere', 'love', 'responsibility']
-----------
['do', 'nt', 'get', 'fooled', 'big', 'name', 'like', 'burt', 'reynolds', 'james', 'wood', 'anne', 'archer', 'they', 'glorified', 'extra', 'their', 'scene', 'probably', 'filmed', 'one', 'day', 'whatever', 'motif', 'movie', 'actor', 'like', 'james', 'wood', 'better', 'make', 'good', 'use', 'to', 'sign', 'bad', 'direction', 'the', 'plot', 'nt', 'bad', 'and', 'acting', 'actor', 'average', 'cuba', 'gooding', 'jr', 'however', 'terrible', 'he', 'unbelievable', 'i', 'almost', 'laughed', 'dramatic', 'scene', 'and', 'since', 'meant', 'serious', 'movie', 'ca', 'nt', 'good', 'thing', 'the', 'action', 'scene', 'bad', 'lacked', 'special', 'punch', 'make', 'exciting', 'again', 'better', 'direction', 'needed', 'also', 'pacin

In [66]:
flat_vocabulary = []
for item in random_elements_v7:
    if isinstance(item, list):
        flat_vocabulary.extend(item)
    else:
        flat_vocabulary.append(item)
flat_vocabulary = list(dict.fromkeys(flat_vocabulary))
print(flat_vocabulary)
# text_rep2 = Pipeline([
#   ('prep', CountVectorizer(vocabulary=flat_vocabulary)), #note we call this vocab when we perform our word count
#   ('rep', TfidfTransformer())]).fit(df_train['text'])
#
# term_vectors = text_rep2.transform(df_train['text']).toarray()
# print(term_vectors)
# print(term_vectors.shape) #if we repeat our experiments, we can see the output dimension is now only size 4 - a value for each word in our vocabulary


['it', 'heartwarming', 'see', 'movie', 'nt', 'bash', 'male', 'in', 'one', 'wifemother', 'leaf', 'family', 'get', 'touch', 'pursue', 'libido', 'the', 'father', 'stay', 'nurture', 'kid', 'letting', 'neither', 'work', 'love', 'life', 'interfere', 'responsibility', 'do', 'fooled', 'big', 'name', 'like', 'burt', 'reynolds', 'james', 'wood', 'anne', 'archer', 'they', 'glorified', 'extra', 'their', 'scene', 'probably', 'filmed', 'day', 'whatever', 'motif', 'actor', 'better', 'make', 'good', 'use', 'to', 'sign', 'bad', 'direction', 'plot', 'and', 'acting', 'average', 'cuba', 'gooding', 'jr', 'however', 'terrible', 'he', 'unbelievable', 'i', 'almost', 'laughed', 'dramatic', 'since', 'meant', 'serious', 'ca', 'thing', 'action', 'lacked', 'special', 'punch', 'exciting', 'again', 'needed', 'also', 'pacing', 'wrong', 'took', 'main', 'character', 'half', 'hour', 'for', 'thriller', '90', 'minute', 'far', 'slow', 'redeeming', 'factor', 'angie', 'harmon', 'she', 'best', 'too', 'director', 'left', 'hang

In [63]:
text_clf = Pipeline([
  ('count', CountVectorizer(max_features=300)),
  ('rep', TfidfTransformer()),
  ('mod', KNeighborsClassifier()),
  ])

# X_train: your training features
# y_train: your training labels (target variable)
# X_test, y_test: your separate test set (not used in cross-validation)
X_train = df_train['text']
y_train = df_train['label']
X_test = df_test['text']
y_test = df_test['label']
acc_score = []

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Split the TRAINING data only
for train_idx, val_idx in kf.split(X_train, y_train):
    # Split training data into train and validation folds
    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    text_clf.fit(X_train, y_train) #we then only fit the training data (note that we oapply the text_clf pipeline object, rather than having to go through each function separately)
    predictions = text_clf.predict(X_test) #and can predict on the test data (similar to above, we can predict using the pipeline directly)
    acc = accuracy_score(predictions, y_test) #we use the accuracy score we imported to give an idea how well the model is doing
    acc_score.append(acc) #we can append it to our list

print("Accuracy:", np.mean(acc_score)) #we can take the mean to get a good overview

Accuracy: 0.66844


In [68]:
from sklearn.base import BaseEstimator, TransformerMixin

class pre_process(BaseEstimator, TransformerMixin):
    def __init__(self):
        # no parameters needed
        pass

    def fit(self, X, y=None):
        # nothing to fit; just return self for compatibility
        return self

    def transform(self, X, y=None):
        prep_text = []
        for x in X:
            # 1) remove HTML
            t = remove_html_tags(x)
            # 2) tokenize
            t = tokenize_text(t)
            # 3) remove punctuation (element‑wise for token lists)
            t = remove_punctuation(t)
            # 4) remove stopwords
            t = remove_stopwords(t)
            # 5) drop empty tokens
            t = remove_empty_elements(t)
            # 6) lowercase all tokens
            t = lowercase_words(t)
            # 7) lemmatize
            t = lemmatize_words(t)

            # Join back to a string for CountVectorizer compatibility
            prep_text.append(" ".join(t))

        return prep_text